# Simulation (ctobssim) 
## (no output on disk but everything is saved in memory)

The purpose of this notebook is to perform a simulation using ctobbsim. In order to to this, one has three options:
- create an observation container file with the [given notebook](observation_definition.ipynb), in which many different pointings can be specified and might be useful for some specific analysis;
- perform a simpler simulation, setting everything directly in this notebook.

In [ ]:
import gammalib
import ctools
import cscripts 

Now that we have created the **observation definition .xml** file, we can run the simulation. 

We only need to provide a **model** for the source that we want to simulate. We have to give a model for the source to be simulated: since many sources are simulated together, the xml model will have a piece for every source to simulate, including the background. An example model is $CTOOLS/share/models/crab.xml.

In [ ]:
#!xmllint $CTOOLS/share/models/crab.xml

## Set up the simulation
The simulation can be set up in two different ways: 

**1) using the gammalib library **

In [ ]:
obs = gammalib.GObservations('out_csobsdef.xml')
obs.models(gammalib.GModels("${CTOOLS}/share/models/crab.xml"))
sim = ctools.ctobssim(obs)
sim.run()

**2) using only ctools**

In [ ]:
sim = ctools.ctobssim()
sim["inobs"] = "out_csobsdef.xml"
sim["inmodel"] = "${CTOOLS}/share/models/crab.xml"   #we load a sample model
sim.run()

...and one can also retreive the results...

In [ ]:
# ...and print the results
#print(sim.obs())
print(sim.obs()[0].events())   #for the first pointing (id = 0001)
print(sim.obs()[1].events())   #for the second pointing (id = 0002)...and so on

**3) using the cscripts named csobsutil**

In [ ]:
pattern = cscripts.obsutils.set_obs_patterns("single", ra=83.63, dec=22.01, offset=1.0)
obs = cscripts.obsutils.set_obs_list(pattern, duration=60, emin=0.05, emax=100.0, rad=5.0, caldb="prod2", irf="South_0.5h")
obs.models(gammalib.GModels("${CTOOLS}/share/models/crab.xml"))
sim = cscripts.obsutils.sim(obs)

...and one can also retreive the results...

In [ ]:
print(sim[0].events())

## View the results
### counts sky map

We create a count map with ctskymap

In [ ]:
skymap = ctools.ctskymap(sim)
skymap['emin'] = 0.1
skymap['emax'] = 100
skymap['nxpix'] = 20
skymap['nypix'] = 20
skymap['binsz'] = 0.5
skymap['coordsys'] = "CEL"
skymap['proj'] = "AIT"
skymap['xref'] = 83.63 #RA
skymap['yref'] = 22.01 #DEC
skymap['bkgsubtract'] = "NONE" 
skymap['outmap'] = "skymap.fits"
skymap.execute()

and we view it with ds9:

In [ ]:
!ds9 -wcs icrs -wcs skyformat degrees skymap.fits

Another option is to view it using matplotlib and the astropy library

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from astropy.io import fits
hdu_list = fits.open("skymap.fits")
image_data = hdu_list[0].data
plt.imshow(image_data, cmap='gray', interpolation = 'sinc')#, vmin=0)
plt.colorbar()
#plt.savefig("out.png")

### Spectrum